In [1]:
using DataFrames;
using Distributions;

In [2]:
df = readtable("/Users/nwelch/prelim/data/infectionDataTest.csv");
dst = readtable("/Users/nwelch/prelim/data/infectedDistancesTest.csv");

# Initial Parameter Values

In [3]:
#priors
mu_a=0.7; mu_b=0.004;
theta_a=0.8; theta_b=10;
sigma_a=0.5; sigma_b=100;

# initial/test values
u = mu_a*mu_b
th = theta_a*theta_b;
s = sigma_a*sigma_b;
t = [14, 10, 6];
tl = maximum(t);

# Function Definitions

## log  Lambda

In [4]:
function getLogLambda(tau, theta, mu, sigma, dst, Tlast=30)
    iLessTlast = find(x-> x<Tlast, tau)
    gaussian = Normal(mu, sigma)
    
    sumLog = 0
    for i = iLessTlast
        sumTauJLessTauI = 0
        jLessI = find(x-> x < tau[i], tau)

        for j = jLessI
            sumTauJLessTauI = sumTauJLessTauI + pdf(gaussian, dst[i,j])
        end
        sumLog = sumLog + log(mu + theta*sumTauJLessTauI)
    end
    sumLog
end

getLogLambda (generic function with 2 methods)

In [5]:
getLogLambda(t, th, u, s, dst, tl)

-8.586772098851577

## $\mu$ log-likelihood Ratio

In [6]:
muPrior = Gamma(mu_a, mu_b)

function llRatio_mu(mu, muStar, theta, sigma, tau, dst, Tlast=30, mu_a=0.7, mu_b=0.004)
    tauLessTIndex = find(x-> x<Tlast, tau)
    
    dmu = mu - muStar
    
    dllTerm1 = sum(tau[tauLessTIndex]*dmu)
    dllTerm2 = length(tauLessTIndex) * Tlast * dmu
    
    dllTerm3a = getLogLambda(tau, theta, mu, sigma, dst, Tlast)
    dllTerm3b = getLogLambda(tau, theta, muStar, sigma, dst, Tlast)
    dllTerm3 = dllTerm3a - dllTerm3b
    
    prior = pdf(muPrior, mu)
    priorStar = pdf(muPrior, muStar)
    
    dll = dllTerm1 + dllTerm2 - dllTerm3 + prior - priorStar
    dll
end

llRatio_mu (generic function with 4 methods)

In [7]:
llRatio_mu(u, 0.01, th, s, t, dst, tl)

95.49074474191153

## $\sigma$ log-likelihood Ratio

* check the  $\le$ for Tlast in paper
* check the sign on the priors

In [112]:
function getSumLessT_sigma(sigma, sigmaStar, mu, tau, theta, dst, Tlast=30.)
    tauLessTIndex = find(x -> x<=Tlast, tau) 
    gaussian = Normal(mu, sigma)
    gaussianStar = Normal(mu, sigmaStar)
    
    sumLessT = 0
    for i = tauLessTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<=tau[i], tau)
        for j=tauJLessTauIIndex
            sum_ij = sum_ij + (tau[i] - tau[j])*(
                pdf(gaussian, dst[i,j]) - pdf(gaussianStar, dst[i,j])) #check signs
        end
        sumLessT = sumLessT + theta*sum_ij
    end
    #sum_ij
end

getSumLessT_sigma (generic function with 2 methods)

In [113]:
getSumLessT_sigma(s, 100., u, t, th, dst, tl)

sum_ij = 0.0
sum_ij = 0.015956310550613827
sum_ij = 0.047860586288295794
sumLessT = 0.38288469030636635
sum_ij = 0.0
sum_ij = 0.015956310550613827
sumLessT = 0.510535174711277
sum_ij = 0.0
sumLessT = 0.510535174711277


In [ ]:
function llRatio_sigma(sigma, sigmaStar, mu, tau, theta, dst, Tlast=30,
        sigma_a=0.5, sigma_b=100)
    
    dllTerm1 = getSumLessT_sigma(sigma, sigmaStar, mu, tau, theta, dst)
    
    dllTerm2 = getSumGreaterT_sigma(sigma, sigmaStar, mu, tau, theta, dst)
    
    dllTerm3 = getLogLambda(tau, theta, mu, sigmaStar, dst, Tlast) -
        getLogLambda(tau, theta, mu, sigma, dst, Tlast)
    
    priorStar = dgamma(sigmaStar, shape=sigma_a, rate=sigma_b)
    prior = dgamma(sigma, shape=sigma_a, rate=sigma_b)
    
    dll = dllTerm1 + dllTerm2 + dllTerm3 + priorStar - prior
    
    dll
end